## **<font color='blue'> Mega Sync Google Drive  </font>**


##__Mount GDrive to /content/drive__
#####Use one of below methods. Mounting with Google Drive File Stream app is more comfortable, but it has some writing issues. So you can avoid them by using Rclone instead.

###__Mount with Google Drive File Stream__

In [ ]:
#@markdown <br><center><img src='https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/Google_Drive_logo.png/600px-Google_Drive_logo.png' height="50" alt="Gdrive-logo"/></center>
#@markdown <center><h3>Mount GDrive to /content/drive</h3></center><br>
MODE = "MOUNT" #@param ["MOUNT", "UNMOUNT"]
#Mount your Gdrive!
from google.colab import drive
drive.mount._DEBUG = False
if MODE == "MOUNT":
  drive.mount('/content/drive', force_remount=True)
elif MODE == "UNMOUNT":
  try:
    drive.flush_and_unmount()
  except ValueError:
    pass
  get_ipython().system_raw("rm -rf /root/.config/Google/DriveFS")

###__Mount with Rclone - (More Accurate) - (Updated)__
#####**- Multiple drives including team drives can be mounted at the same runtime session**

In [ ]:
#@markdown <br><center><img src='https://rclone.org/img/logo_on_light__horizontal_color.svg' height="50" alt="Gdrive-logo"/></center>
#@markdown <center><h3>Mount GDrive to /content/drive_name</h3></center><br>
drive_name = "" #@param {type:"string"}
#@markdown If you want to configure this as a team drive, check the below checkbox
team_drive = False #@param {type:"boolean"}

import subprocess, time, json, os
from google.colab import output
from IPython.display import HTML, clear_output

def formatx(i, j, k):
    val=auth_response[i].split(": ")[1][1-j:-2+j+k]
    return val

rclone_status = subprocess.check_output("command -v rclone &> /dev/null",  shell=True).decode()
if rclone_status:
  pass
else:
  !curl https://rclone.org/install.sh | sudo bash
  !rclone config file
  !touch /root/.config/rclone/rclone.conf
  output.clear()
  display(HTML("<h2 style=\"font-family:Trebuchet MS;color:#446785;\">Rclone: Installed successfully .</h2><br>"))
  time.sleep(1)
  display(HTML("<h2 style=\"font-family:Trebuchet MS;color:#446785;\">Next: Drive authentication...</h2><br>"))
  time.sleep(2)
  output.clear()

client_id="202264815644.apps.googleusercontent.com"
client_secret="X4Z3ca8xfWDb1Voo-F9a7ZxJ"
print("Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id={}&redirect_uri=urn:ietf:wg:oauth:2.0:oob&scope=https://www.googleapis.com/auth/drive&response_type=code".format(client_id))
auth_code=input("Enter your authorization code: ")
auth_response = subprocess.check_output("curl -d client_id={} -d client_secret={} -d grant_type=authorization_code -d redirect_uri=urn:ietf:wg:oauth:2.0:oob -d code={} https://accounts.google.com/o/oauth2/token".format(client_id, client_secret, auth_code), shell=True).decode().split("\n")[1:6]
access_token, expires_in, refresh_token, scope, token_type = formatx(0, 0, 0), formatx(1, 1, 0), formatx(2, 0, 0), formatx(3, 0, 0), formatx(4, 0, 1)
token_in_json = "{"+", ".join("".join("".join(auth_response).split("  ")).split(","))+"}"

output.clear()
display(HTML("<h2 style=\"font-family:Trebuchet MS;color:#446785;\">Authenticated successfully.</h2><br>"))
time.sleep(2)
output.clear()

if team_drive:
  print("Fetching team drive list...")
  check_drives = subprocess.check_output('''curl \
  'https://www.googleapis.com/drive/v3/drives' \
  --header 'Authorization: Bearer {}' \
  --header 'Accept: application/json' \
  --compressed'''.format(access_token),  shell=True).decode()
  drive_list = json.loads(check_drives).get('drives')
  print("Choose a number from below.(If you want to use the 1st team drive from the below list, type 1 and press enter)")
  for i in range(len(drive_list)):
    print(i+1,"/", drive_list[i].get('name'))
    print("  \\","\""+drive_list[i].get('id')+"\"")
  selected_num = int(input("Enter the number : "))
  root_id = drive_list[selected_num-1].get('id')
else:
  print("Fetching root id...")
  check_about = subprocess.check_output('''curl \
  'https://www.googleapis.com/drive/v2/about' \
  --header 'Authorization: Bearer {}' \
  --header 'Accept: application/json' \
  --compressed'''.format(access_token),  shell=True).decode()
  root_id = json.loads(check_about).get('rootFolderId')

#read file
rfile = open(r"/root/.config/rclone/rclone.conf", "r")
rfile_content = rfile.readlines()
rfile.close()
#write file
wfile = open(r"/root/.config/rclone/rclone.conf","a+")
if len(rfile_content) > 0:
  wfile.write("\n\n")
wfile.write("[{}]\ntype = drive\nscope = drive\ntoken = {}\n".format(drive_name, token_in_json))
if team_drive:
  wfile.write("team_drive = {}".format(root_id))
else:
  wfile.write("root_folder_id = {}".format(root_id))
wfile.close()

output.clear()
display(HTML("<h2 style=\"font-family:Trebuchet MS;color:#446785;\">Configuration: Completed successfully</h2><br>"))
time.sleep(1)
display(HTML("<h2 style=\"font-family:Trebuchet MS;color:#446785;\">Next: Mounting drive...</h2><br>"))
time.sleep(2)
output.clear()

rclone_config_name = drive_name
local_mount_location = "/content/"+drive_name+"/"

# clear nohup
open("nohup.out", 'w').close()

# unmount first
#!fusermount -u $local_mount_location 2>/dev/null

# mount without waiting for the command to complete
!mkdir $local_mount_location 2>/dev/null
!nohup rclone mount $rclone_config_name: $local_mount_location --buffer-size 96M &

# Show the output that was written to nohup
time.sleep(3)
f = open(r"nohup.out", "r")
nohupText = f.read()
f.close()

dirs = os.listdir(local_mount_location)

if len(dirs) > 0:
  clear_output(wait=True)
  print("Succeeded. ", str(len(dirs)), "dirs found at", local_mount_location)

else:
  print("\n\nNot succeeded. No files or directories in mounted location. \nCheck your config name and content. If the rclone command was not found, run the cell above.\n\n")
  print("log:\n", nohupText)

##__Transfer from Mega to GDrive__





In [ ]:
import sys, os, urllib.request
import time
import subprocess
import contextlib
from IPython.display import clear_output
#@markdown <br><center><img src='https://mega.nz/favicon.ico?v=3' height="50" alt="MEGA-logo"/></center>
#@markdown <center><h2>Transfer from Mega to GDrive</h2></center><br>

def run_command(cmd, capture_output=True, timeout=60):
    """Run command with timeout and proper error handling"""
    try:
        result = subprocess.run(cmd, shell=True, capture_output=capture_output, 
                              text=True, timeout=timeout)
        return result
    except subprocess.TimeoutExpired:
        print(f"⏰ Command timed out: {cmd}")
        return None
    except Exception as e:
        print(f"❌ Command failed: {cmd} - {e}")
        return None

def detect_system_info():
    """Detect the system architecture and OS version for Google Colab"""
    try:
        # Get OS info
        os_info = run_command('cat /etc/os-release')
        arch_info = run_command('uname -m')
        
        os_version = "Ubuntu_22.04"  # Google Colab default
        arch = "x86_64"  # Google Colab default
        
        if os_info and os_info.stdout:
            if "Ubuntu" in os_info.stdout:
                if "22.04" in os_info.stdout:
                    os_version = "Ubuntu_22.04"
                elif "20.04" in os_info.stdout:
                    os_version = "Ubuntu_20.04"
                elif "18.04" in os_info.stdout:
                    os_version = "Ubuntu_18.04"
        
        if arch_info and arch_info.stdout:
            arch = arch_info.stdout.strip()
            
        print(f"🔍 Detected system: {os_version} ({arch})")
        return os_version, arch
        
    except Exception as e:
        print(f"⚠️  Using defaults for Google Colab: Ubuntu_22.04 (x86_64)")
        return "Ubuntu_22.04", "x86_64"

def install_megacmd_official():
    """Install MEGAcmd using official MEGA download URLs"""
    print("🔧 Installing MEGAcmd from official MEGA sources...")
    
    # Detect system
    os_version, arch = detect_system_info()
    
    try:
        # Install dependencies first
        print("📦 Installing dependencies...")
        deps_cmd = 'sudo apt-get update && sudo apt-get install -y wget curl apt-transport-https ca-certificates'
        run_command(deps_cmd)
        
        # Official download URLs from mega.nz/cmd (based on the screenshot)
        download_urls = {
            "Ubuntu_22.04": {
                "x86_64": "https://mega.nz/linux/repo/Ubuntu_22.04/amd64/megacmd_Ubuntu_22.04_amd64.deb"
            },
            "Ubuntu_20.04": {
                "x86_64": "https://mega.nz/linux/repo/Ubuntu_20.04/amd64/megacmd_Ubuntu_20.04_amd64.deb"
            },
            "Debian_12": {
                "x86_64": "https://mega.nz/linux/repo/Debian_12/amd64/megacmd_Debian_12_amd64.deb"
            }
        }
        
        # Get the appropriate URL
        url = None
        if os_version in download_urls and arch in download_urls[os_version]:
            url = download_urls[os_version][arch]
        
        # Fallback URLs for Google Colab
        fallback_urls = [
            "https://mega.nz/linux/repo/Ubuntu_22.04/amd64/megacmd_Ubuntu_22.04_amd64.deb",
            "https://mega.nz/linux/repo/Ubuntu_20.04/amd64/megacmd_Ubuntu_20.04_amd64.deb",
            "https://mega.nz/linux/repo/Debian_12/amd64/megacmd_Debian_12_amd64.deb"
        ]
        
        urls_to_try = [url] + fallback_urls if url else fallback_urls
        
        for download_url in urls_to_try:
            if not download_url:
                continue
                
            print(f"🔄 Downloading: {download_url}")
            
            # Download the package
            download_result = run_command(f'wget -q --timeout=30 -O /tmp/megacmd.deb "{download_url}"', timeout=60)
            
            if download_result and download_result.returncode == 0:
                # Verify it's a valid deb package
                verify_result = run_command('file /tmp/megacmd.deb')
                
                if verify_result and 'Debian binary package' in verify_result.stdout:
                    print("📦 Valid package downloaded, installing...")
                    
                    # Install the package
                    install_result = run_command('sudo dpkg -i /tmp/megacmd.deb', timeout=120)
                    
                    if install_result and install_result.returncode == 0:
                        print("✅ MEGAcmd installed successfully!")
                        return True
                    else:
                        print("🔧 Fixing dependencies and retrying...")
                        run_command('sudo apt-get install -f -y')
                        
                        # Try installing again after fixing deps
                        install_result2 = run_command('sudo dpkg -i /tmp/megacmd.deb', timeout=120)
                        if install_result2 and install_result2.returncode == 0:
                            print("✅ MEGAcmd installed after fixing dependencies!")
                            return True
                else:
                    print("❌ Downloaded file is not a valid Debian package")
                    # Try next URL
                    continue
            else:
                print("❌ Download failed, trying next URL...")
                continue
        
        print("❌ All official download URLs failed")
        return False
        
    except Exception as e:
        print(f"❌ MEGAcmd installation error: {e}")
        return False

def install_megatools():
    """Install megatools as a reliable fallback"""
    print("🔧 Installing megatools as fallback...")
    
    # Update and install
    run_command('sudo apt-get update -qq')
    result = run_command('sudo apt-get install -y megatools')
    
    if result and result.returncode == 0:
        print("✅ Megatools installed successfully!")
        return True
    
    # Try with universe repository
    print("🔄 Trying with universe repository...")
    run_command('sudo add-apt-repository universe -y')
    run_command('sudo apt-get update -qq')
    result = run_command('sudo apt-get install -y megatools')
    
    if result and result.returncode == 0:
        print("✅ Megatools installed from universe!")
        return True
        
    return False

def check_mega_tools():
    """Check which MEGA tools are available"""
    tools = ['megadl', 'megaget', 'mega-get', 'mega-cmd']
    available = []
    
    for tool in tools:
        result = run_command(f'which {tool}')
        if result and result.returncode == 0:
            available.append(tool)
    
    return available

def download_with_megatools(url, output_path):
    """Download using megatools"""
    print(f"📥 Downloading with megatools: {url}")
    
    # Create output directory
    os.makedirs(output_path, exist_ok=True)
    
    try:
        # Use megadl for downloading
        cmd = f'megadl --path "{output_path}" "{url}"'
        print(f"🔧 Running: {cmd}")
        
        # Run with live output
        process = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, 
                                 stderr=subprocess.STDOUT, universal_newlines=True)
        
        for line in iter(process.stdout.readline, ''):
            if line:
                print(line.strip())
        
        process.wait()
        
        if process.returncode == 0:
            print("✅ Download completed successfully!")
            return True
        else:
            print("❌ Download failed!")
            return False
            
    except Exception as e:
        print(f"❌ Download error: {e}")
        return False

def download_with_megacmd(url, output_path):
    """Download using MEGAcmd"""
    print(f"📥 Downloading with MEGAcmd: {url}")
    
    try:
        # Create output directory
        os.makedirs(output_path, exist_ok=True)
        
        # Kill any existing processes and start fresh
        run_command('pkill -f mega-cmd')
        time.sleep(1)
        
        # Start daemon
        print("🚀 Starting MEGA daemon...")
        run_command('mega-cmd-server > /dev/null 2>&1 &')
        time.sleep(3)
        
        # Download
        print("⬇️  Starting download...")
        cmd = f'mega-get "{url}" "{output_path}"'
        result = run_command(cmd, timeout=600)  # 10 minute timeout
        
        if result and result.returncode == 0:
            print("✅ Download completed!")
            return True
        else:
            print(f"❌ Download failed: {result.stderr if result else 'Unknown error'}")
            return False
            
    except Exception as e:
        print(f"❌ MEGAcmd download error: {e}")
        return False

# Configuration
URL = ""  #@param {type:"string"}
OUTPUT_PATH = "downloads"  #@param {type:"string"}

#@markdown ---
#@markdown **Instructions:**
#@markdown - **URL**: MEGA public link to download (e.g., https://mega.nz/file/...)
#@markdown - **OUTPUT_PATH**: Local folder where files will be downloaded
#@markdown ---

print("🚀 Setting up MEGA tools for download...")
print("🖥️  Google Colab runs Ubuntu 22.04 LTS x86_64")

# Check available tools
available_tools = check_mega_tools()
print(f"🔍 Found existing tools: {available_tools}")

# Install tools if nothing is available
if not available_tools:
    print("\n📦 No MEGA tools found. Installing...")
    
    # Try MEGAcmd first (more features)
    if install_megacmd_official():
        available_tools = check_mega_tools()
        print("✅ MEGAcmd installation successful!")
    else:
        print("⚠️  MEGAcmd failed, trying megatools...")
        if install_megatools():
            available_tools = check_mega_tools()
            print("✅ Megatools installation successful!")

# Check final status
megatools_available = any(tool in available_tools for tool in ['megadl', 'megaget'])
megacmd_available = any(tool in available_tools for tool in ['mega-get', 'mega-cmd'])

print(f"\n📋 Final tool status:")
print(f"  MEGAcmd: {'✅ Available' if megacmd_available else '❌ Not available'}")
print(f"  Megatools: {'✅ Available' if megatools_available else '❌ Not available'}")

# Main execution
if not URL:
    print("\n❌ Please enter a MEGA URL to download")
    print("📎 Example: https://mega.nz/file/abc123/xyz456")
    
elif not URL.startswith('https://mega.nz/'):
    print("\n❌ Invalid MEGA URL format")
    print("📎 URL should start with: https://mega.nz/")
    
elif not megatools_available and not megacmd_available:
    print("\n❌ No MEGA tools available!")
    print("\n🛠️  Manual installation commands for Google Colab:")
    print("!wget https://mega.nz/linux/repo/Ubuntu_22.04/amd64/megacmd_Ubuntu_22.04_amd64.deb")
    print("!sudo dpkg -i megacmd_Ubuntu_22.04_amd64.deb")
    print("!sudo apt-get install -f -y")  # Fix dependencies if needed
    print("\nOr try: !sudo apt-get update && sudo apt-get install -y megatools")
    
else:
    print(f"\n🎯 Starting download:")
    print(f"🔗 URL: {URL}")
    print(f"📁 Output: {OUTPUT_PATH}")
    
    download_success = False
    
    # Try MEGAcmd first (if available)
    if megacmd_available and not download_success:
        print("\n🔧 Attempting download with MEGAcmd...")
        download_success = download_with_megacmd(URL, OUTPUT_PATH)
    
    # Fallback to megatools
    if megatools_available and not download_success:
        print("\n🔧 Attempting download with megatools...")
        download_success = download_with_megatools(URL, OUTPUT_PATH)
    
    # Final status and results
    if download_success:
        print("\n🎉 DOWNLOAD COMPLETED SUCCESSFULLY!")
        print(f"📁 Files saved to: {OUTPUT_PATH}")
        
        # List downloaded files
        try:
            if os.path.exists(OUTPUT_PATH):
                files = os.listdir(OUTPUT_PATH)
                if files:
                    print(f"\n📋 Downloaded files ({len(files)}):")
                    for f in files[:10]:  # Show first 10 files
                        file_path = os.path.join(OUTPUT_PATH, f)
                        if os.path.isfile(file_path):
                            size = os.path.getsize(file_path)
                            size_mb = size / (1024*1024)
                            print(f"  📄 {f} ({size_mb:.1f} MB)")
                        else:
                            print(f"  📁 {f}/")
                    if len(files) > 10:
                        print(f"  ... and {len(files) - 10} more items")
                else:
                    print("⚠️  No files found in output directory")
            else:
                print("⚠️  Output directory was not created")
        except Exception as e:
            print(f"⚠️  Could not list files: {e}")
    else:
        print("\n💥 DOWNLOAD FAILED!")
        print("🔧 Troubleshooting steps:")
        print("1. ✅ Check that the MEGA URL is correct and accessible")
        print("2. 💾 Ensure you have enough disk space")
        print("3. 🌐 Verify your internet connection is stable")
        print("4. 🔄 Try running the cell again")

print(f"\n{'='*60}")
print("📥 MEGA to Google Drive Transfer Tool")
print(f"{'='*60}")

####_Update: From google drive to mega_

In [ ]:
# @markdown <center><h3>Upload from Google Drive to Mega</h3></center><br>
import os
import subprocess
import time
import sys
from pathlib import Path


def run_command(cmd, capture_output=True, timeout=60):
    """Run command with timeout and proper error handling"""
    try:
        result = subprocess.run(cmd, shell=True, capture_output=capture_output, 
                              text=True, timeout=timeout)
        return result
    except subprocess.TimeoutExpired:
        print(f"⏰ Command timed out: {cmd}")
        return None
    except Exception as e:
        print(f"❌ Command failed: {cmd} - {e}")
        return None

def detect_system_info():
    """Detect the system architecture and OS version"""
    try:
        # Get OS info
        os_info = run_command('cat /etc/os-release')
        arch_info = run_command('uname -m')
        
        os_version = "Unknown"
        arch = "x86_64"
        
        if os_info and os_info.stdout:
            if "Ubuntu" in os_info.stdout:
                if "22.04" in os_info.stdout:
                    os_version = "Ubuntu_22.04"
                elif "20.04" in os_info.stdout:
                    os_version = "Ubuntu_20.04"
                elif "18.04" in os_info.stdout:
                    os_version = "Ubuntu_18.04"
                else:
                    os_version = "Ubuntu_22.04"  # Default to latest
            elif "Debian" in os_info.stdout:
                if "12" in os_info.stdout:
                    os_version = "Debian_12"
                elif "11" in os_info.stdout:
                    os_version = "Debian_11.0"
                else:
                    os_version = "Debian_12"  # Default to latest
        
        if arch_info and arch_info.stdout:
            arch = arch_info.stdout.strip()
            
        print(f"🔍 Detected system: {os_version} ({arch})")
        return os_version, arch
        
    except Exception as e:
        print(f"⚠️  Could not detect system info: {e}")
        return "Ubuntu_22.04", "x86_64"  # Google Colab default

def install_megacmd_official():
    """Install MEGAcmd using official MEGA repositories and direct downloads"""
    print("🔧 Installing MEGAcmd from official MEGA sources...")
    
    # Detect system
    os_version, arch = detect_system_info()
    
    try:
        # First, install dependencies
        print("📦 Installing dependencies...")
        deps_cmd = 'sudo apt-get update && sudo apt-get install -y wget curl apt-transport-https ca-certificates'
        run_command(deps_cmd)
        
        # Method 1: Try official MEGA .deb packages based on detected system
        print(f"🌐 Downloading MEGAcmd for {os_version} {arch}...")
        
        # Official download URLs from mega.nz/cmd
        download_urls = {
            "Ubuntu_22.04": {
                "x86_64": "https://mega.nz/linux/repo/Ubuntu_22.04/amd64/megacmd_Ubuntu_22.04_amd64.deb",
                "aarch64": "https://mega.nz/linux/repo/Ubuntu_22.04/arm64/megacmd_Ubuntu_22.04_arm64.deb"
            },
            "Ubuntu_20.04": {
                "x86_64": "https://mega.nz/linux/repo/Ubuntu_20.04/amd64/megacmd_Ubuntu_20.04_amd64.deb"
            },
            "Debian_12": {
                "x86_64": "https://mega.nz/linux/repo/Debian_12/amd64/megacmd_Debian_12_amd64.deb"
            },
            "Debian_11.0": {
                "x86_64": "https://mega.nz/linux/repo/Debian_11.0/amd64/megacmd_Debian_11.0_amd64.deb"
            }
        }
        
        # Try to get the right URL for the system
        url = None
        if os_version in download_urls:
            if arch in download_urls[os_version]:
                url = download_urls[os_version][arch]
            elif "x86_64" in download_urls[os_version]:
                url = download_urls[os_version]["x86_64"]
                print(f"⚠️  Using x86_64 package for {arch} architecture")
        
        # Fallback URLs if detection fails
        fallback_urls = [
            "https://mega.nz/linux/repo/Ubuntu_22.04/amd64/megacmd_Ubuntu_22.04_amd64.deb",
            "https://mega.nz/linux/repo/Ubuntu_20.04/amd64/megacmd_Ubuntu_20.04_amd64.deb",
            "https://mega.nz/linux/repo/Debian_12/amd64/megacmd_Debian_12_amd64.deb"
        ]
        
        urls_to_try = [url] if url else fallback_urls
        urls_to_try.extend(fallback_urls)
        
        for download_url in urls_to_try:
            if not download_url:
                continue
                
            print(f"🔄 Trying: {download_url}")
            
            # Download the package
            result = run_command(f'wget -q --timeout=30 -O /tmp/megacmd.deb "{download_url}"', timeout=60)
            
            if result and result.returncode == 0:
                # Verify it's a valid deb file
                verify_result = run_command('file /tmp/megacmd.deb')
                
                if verify_result and 'Debian binary package' in verify_result.stdout:
                    print("📦 Valid package downloaded, installing...")
                    
                    # Install the package
                    install_result = run_command('sudo dpkg -i /tmp/megacmd.deb', timeout=120)
                    
                    if install_result and install_result.returncode == 0:
                        print("✅ MEGAcmd installed successfully!")
                        return True
                    else:
                        print("🔧 Fixing dependencies and retrying...")
                        run_command('sudo apt-get install -f -y')
                        
                        # Try installing again
                        install_result2 = run_command('sudo dpkg -i /tmp/megacmd.deb', timeout=120)
                        if install_result2 and install_result2.returncode == 0:
                            print("✅ MEGAcmd installed after fixing dependencies!")
                            return True
                else:
                    print("❌ Downloaded file is not a valid Debian package")
            else:
                print("❌ Download failed for this URL")
        
        # Method 2: Try using official installation script if available
        print("🔄 Trying official installation script...")
        script_result = run_command('curl -O https://mega.nz/linux/MEGAsync/apt.sh && sudo bash apt.sh')
        if script_result and script_result.returncode == 0:
            # Try installing megacmd after setting up repo
            install_result = run_command('sudo apt-get update && sudo apt-get install -y megacmd')
            if install_result and install_result.returncode == 0:
                print("✅ MEGAcmd installed via official script!")
                return True
        
        return False
        
    except Exception as e:
        print(f"❌ MEGAcmd installation error: {e}")
        return False

def install_megatools():
    """Install megatools - a reliable MEGA client"""
    print("🔧 Installing megatools (reliable alternative)...")
    
    # Update package lists
    result = run_command('sudo apt-get update -qq')
    if result and result.returncode != 0:
        print("⚠️  Package update failed, continuing anyway...")
    
    # Install megatools
    result = run_command('sudo apt-get install -y megatools')
    if result and result.returncode == 0:
        print("✅ Megatools installed successfully!")
        return True
    
    # If regular installation fails, try with universe repository
    print("🔄 Enabling universe repository and retrying...")
    run_command('sudo add-apt-repository universe -y')
    run_command('sudo apt-get update -qq')
    result = run_command('sudo apt-get install -y megatools')
    if result and result.returncode == 0:
        print("✅ Megatools installed from universe repository!")
        return True
        
    # Try installing from snap as last resort
    print("🔄 Trying snap installation...")
    result = run_command('sudo snap install megatools')
    if result and result.returncode == 0:
        print("✅ Megatools installed via snap!")
        return True
    
    return False

def check_mega_tools():
    """Check which MEGA tools are available"""
    tools = {
        'megatools': ['megals', 'megadl', 'megaput', 'megamkdir'],
        'megacmd': ['mega-cmd', 'mega-login', 'mega-put', 'mega-ls']
    }
    
    available = {}
    
    for tool_type, commands in tools.items():
        available[tool_type] = []
        for cmd in commands:
            result = run_command(f'which {cmd}')
            if result and result.returncode == 0:
                available[tool_type].append(cmd)
    
    return available

def setup_megatools_config(username, password):
    """Setup megatools configuration"""
    config_content = f"""[Login]
Username = {username}
Password = {password}
"""
    
    # Create config directory
    config_dir = os.path.expanduser("~/.megarc")
    try:
        with open(config_dir, 'w') as f:
            f.write(config_content)
        print("✅ Megatools configuration created")
        return True
    except Exception as e:
        print(f"❌ Failed to create megatools config: {e}")
        return False

def upload_with_megatools(local_path, remote_path, username, password):
    """Upload using megatools"""
    print(f"📤 Uploading with megatools: {local_path} -> {remote_path}")
    
    # Setup configuration
    if not setup_megatools_config(username, password):
        return False
    
    try:
        if os.path.isfile(local_path):
            # Upload single file
            cmd = f'megaput --path "{remote_path}" "{local_path}"'
            result = run_command(cmd, timeout=300)  # 5 minute timeout
            
            if result and result.returncode == 0:
                print("✅ File uploaded successfully!")
                return True
            else:
                print(f"❌ Upload failed: {result.stderr if result else 'Unknown error'}")
                return False
                
        elif os.path.isdir(local_path):
            # Upload directory recursively
            success_count = 0
            total_files = 0
            
            for root, dirs, files in os.walk(local_path):
                for file in files:
                    total_files += 1
                    local_file = os.path.join(root, file)
                    
                    # Create relative path for remote
                    rel_path = os.path.relpath(local_file, local_path)
                    remote_file = f"{remote_path.rstrip('/')}/{rel_path}".replace('\\', '/')
                    
                    print(f"📄 Uploading: {rel_path}")
                    
                    # Create remote directory if needed
                    remote_dir = os.path.dirname(remote_file).replace('\\', '/')
                    if remote_dir and remote_dir != remote_path:
                        run_command(f'megamkdir -p "{remote_dir}"')
                    
                    # Upload file
                    cmd = f'megaput --path "{remote_file}" "{local_file}"'
                    result = run_command(cmd, timeout=300)
                    
                    if result and result.returncode == 0:
                        success_count += 1
                        print(f"  ✅ Uploaded: {rel_path}")
                    else:
                        print(f"  ❌ Failed: {rel_path}")
            
            print(f"📊 Upload completed: {success_count}/{total_files} files")
            return success_count > 0
            
    except Exception as e:
        print(f"❌ Upload error: {e}")
        return False

def upload_with_megacmd(local_path, remote_path, username, password):
    """Upload using MEGAcmd"""
    print(f"📤 Uploading with MEGAcmd: {local_path} -> {remote_path}")
    
    try:
        # Kill any existing processes
        run_command('pkill -f mega-cmd')
        time.sleep(1)
        
        # Start mega-cmd daemon
        print("🚀 Starting MEGA daemon...")
        daemon_result = run_command('mega-cmd-server > /dev/null 2>&1 &')
        time.sleep(3)
        
        # Login
        print("🔐 Logging in...")
        login_cmd = f'mega-login "{username}" "{password}"'
        result = run_command(login_cmd, timeout=30)
        
        if not result or result.returncode != 0:
            # Check if already logged in
            whoami_result = run_command('mega-whoami')
            if whoami_result and whoami_result.returncode == 0 and username in whoami_result.stdout:
                print("✅ Already logged in!")
            else:
                print("❌ MEGAcmd login failed")
                return False
        else:
            print("✅ MEGAcmd login successful")
        
        # Upload
        print("⬆️  Starting upload...")
        upload_cmd = f'mega-put "{local_path}" "{remote_path}"'
        result = run_command(upload_cmd, timeout=600)  # 10 minute timeout
        
        if result and result.returncode == 0:
            print("✅ Upload successful!")
            return True
        else:
            print(f"❌ Upload failed: {result.stderr if result else 'Unknown error'}")
            return False
            
    except Exception as e:
        print(f"❌ MEGAcmd upload error: {e}")
        return False

# Main installation and setup
print("🚀 Setting up MEGA tools for Google Drive to Mega transfer...")
print("🖥️  Google Colab typically runs Ubuntu 22.04 LTS x86_64")

# Check what's already available
available_tools = check_mega_tools()
print(f"🔍 Currently available: {available_tools}")

# Install tools if needed
megatools_available = len(available_tools.get('megatools', [])) > 0
megacmd_available = len(available_tools.get('megacmd', [])) > 0

if not megatools_available and not megacmd_available:
    print("\n📦 No MEGA tools found. Installing...")
    
    # Try MEGAcmd first with official packages
    if install_megacmd_official():
        megacmd_available = True
        print("✅ MEGAcmd installed successfully!")
    else:
        print("⚠️  MEGAcmd installation failed, trying megatools...")
        if install_megatools():
            megatools_available = True
            print("✅ Megatools installed as fallback!")

# Re-check what's available after installation
available_tools = check_mega_tools()
megatools_available = len(available_tools.get('megatools', [])) > 0
megacmd_available = len(available_tools.get('megacmd', [])) > 0

print(f"\n🔧 Final tool status:")
print(f"  MEGAcmd: {'✅ Available' if megacmd_available else '❌ Not available'}")
print(f"  Megatools: {'✅ Available' if megatools_available else '❌ Not available'}")

# Configuration parameters
USERNAME = ""  # @param {type:"string"}
PASSWORD = ""  # @param {type:"string"}
GDRIVE_PATH = ""  # @param {type:"string"}
MEGA_DESTINATION = "/"  # @param {type:"string"}

#@markdown ---
#@markdown **Instructions:**
#@markdown - **USERNAME**: Your Mega account email
#@markdown - **PASSWORD**: Your Mega account password  
#@markdown - **GDRIVE_PATH**: Path to file/folder in mounted Google Drive (e.g., "/content/drive/MyDrive/my_folder")
#@markdown - **MEGA_DESTINATION**: Destination folder in Mega (e.g., "/" for root, "/uploads" for uploads folder)
#@markdown ---

# Main execution
if not megatools_available and not megacmd_available:
    print("\n❌ CRITICAL ERROR: No MEGA tools could be installed!")
    print("\n🛠️  Manual Installation Options:")
    print("1. Try: !wget https://mega.nz/linux/repo/Ubuntu_22.04/amd64/megacmd_Ubuntu_22.04_amd64.deb && sudo dpkg -i megacmd_Ubuntu_22.04_amd64.deb")
    print("2. Or: !sudo apt-get update && sudo apt-get install -y megatools")
    print("3. Check system info with: !cat /etc/os-release && uname -m")
    
elif not USERNAME or not PASSWORD:
    print("\n❌ Please enter your MEGA credentials:")
    print("📧 USERNAME: Your Mega account email")
    print("🔐 PASSWORD: Your Mega account password")
    
elif not GDRIVE_PATH:
    print("\n❌ Please enter your Google Drive path:")
    print("📁 Example: /content/drive/MyDrive/folder_to_upload")
    print("💡 Make sure Google Drive is mounted first!")
    
elif not os.path.exists(GDRIVE_PATH):
    print(f"\n❌ Path does not exist: {GDRIVE_PATH}")
    print("🔍 Please check:")
    print("1. Is Google Drive mounted?")
    print("2. Is the path spelled correctly?")
    print("3. Does the file/folder exist?")
    
else:
    print(f"\n🎯 Starting upload:")
    print(f"📁 From: {GDRIVE_PATH}")
    print(f"☁️  To: {MEGA_DESTINATION}")
    print(f"👤 Account: {USERNAME}")
    
    # Try upload with available tools - prioritize MEGAcmd
    upload_success = False
    
    if megacmd_available and not upload_success:
        print("\n🔧 Attempting upload with MEGAcmd...")
        upload_success = upload_with_megacmd(GDRIVE_PATH, MEGA_DESTINATION, USERNAME, PASSWORD)
    
    if megatools_available and not upload_success:
        print("\n🔧 Attempting upload with megatools...")
        upload_success = upload_with_megatools(GDRIVE_PATH, MEGA_DESTINATION, USERNAME, PASSWORD)
    
    # Final status
    if upload_success:
        print("\n🎉 UPLOAD COMPLETED SUCCESSFULLY!")
        print("✅ Your files have been transferred to MEGA")
        
        # Show account status if possible
        if megacmd_available:
            print("\n📊 Checking your MEGA account...")
            result = run_command('mega-ls')
            if result and result.returncode == 0:
                print("📁 Your MEGA files:")
                print(result.stdout)
        elif megatools_available:
            result = run_command('megals')
            if result and result.returncode == 0:
                print("📁 Your MEGA files:")
                print(result.stdout)
    else:
        print("\n💥 UPLOAD FAILED!")
        print("🔧 Try checking:")
        print("1. Your MEGA credentials are correct")
        print("2. You have enough storage space")
        print("3. Your internet connection is stable")
        print("4. The file paths are correct")

print(f"\n{'='*70}")
print("🔄 Google Drive ↔️ MEGA Transfer Tool")
print(f"{'='*70}")